In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 3.8MB 44.5MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [52]:
# 토큰화를 위한 모듈
from konlpy.tag import Okt
import pandas as pd
import re

pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

# 네이버 웹툰 가져오기
naver_wt = pd.read_csv('./drive/My Drive/acyn/naver_webtoon.csv', delimiter = '\t', header=None, encoding='utf-8')
naver_wt = pd.DataFrame({'name' : naver_wt[0].str.split(',').str[0], 'intro' : naver_wt[0], 'url' : naver_wt[0].str.split(',').str[-1] }, columns = ['name', 'intro', 'url'])

# 다음 웹툰 가져오기
daum_wt = pd.read_csv('./drive/My Drive/acyn/daum_webtoon.csv', delimiter = '\t', header=None, encoding='utf-8')
daum_wt = pd.DataFrame({'name' : daum_wt[0].str.split(',').str[0], 'intro' : daum_wt[0], 'url' : daum_wt[0].str.split(',').str[-1]}, columns = ['name', 'intro' , 'url'])

# 네이버 웹소설 가져오기
naver_nvl = pd.read_csv('./drive/My Drive/acyn/naver_novel.csv', header=None, encoding='utf-8')
naver_nvl = pd.DataFrame({'name' : naver_nvl[0], 'intro' : naver_nvl[0]+naver_nvl[1], 'url' : naver_nvl[2]}, columns = ['name', 'intro', 'url'])

# 넷플릭스 가져오기
netflix = pd.read_csv('./drive/My Drive/acyn/netflix.csv')
netflix = netflix.dropna(axis=0)
# netflix = netflix.drop(['Unnamed: 0'], axis=1)
netflix = netflix.loc[:,['title', 'netflix', 'link']]

# stopword 제거
sw = ['.', ',', '"','"&', '"""','\n', '이', '가', '을', '를', '는', '은', '과', '의', '로','에서','에', '도','에게','와', '인']

# 네이버 웹툰 토큰화 및 stopword 제거
for intro_num in range(len(naver_wt.intro)):
    words = re.compile("[^\w]").sub(' ', naver_wt.intro[intro_num])
    words = Okt().morphs(words)
    sw_removed = []
    for i in words:
       if i.lower() not in sw:
           sw_removed.append(i)
    naver_wt.intro[intro_num] = sw_removed

# 다음 웹툰 토큰화 및 stopword 제거
for intro_num in range(len(daum_wt.intro)):
    words = re.compile("[^\w]").sub(' ', daum_wt.intro[intro_num])
    words = Okt().morphs(words)
    sw_removed = []
    for i in words:
       if i.lower() not in sw:
           sw_removed.append(i)
    daum_wt.intro[intro_num] = sw_removed

# 네이버 웹소설 토큰화 및 stopword 제거
for intro_num in range(len(naver_nvl.intro)):
    words = re.compile("[^\w]").sub(' ', naver_nvl.intro[intro_num])
    words = Okt().morphs(words)
    sw_removed = []
    for i in words:
       if i.lower() not in sw:
           sw_removed.append(i)
    naver_nvl.intro[intro_num] = sw_removed

# 넷플릭스 토큰화 및 stopword 제거
toknized_plots_list = []
for plot in netflix.netflix:
  words_list_for_one_plot = Okt().morphs(plot)
  sw_removed = []
  for i in words_list_for_one_plot:
      if i.lower() not in sw:
          sw_removed.append(i)
  toknized_plots_list.append(sw_removed)
  
netflix.netflix = toknized_plots_list

In [53]:
# 사용자 입력받기
input_wd = input()

naver_wt_recommend = []
naver_wt_url = []

daum_wt_recommend = []
daum_wt_url = []

naver_nvl_recommend = []
naver_nvl_url = []

netflix_recommend = []
netflix_url = []

# 네이버 웹툰에서 입력 관련 컨텐츠 뽑기
for intro_num in range(len(naver_wt.intro)):
    if input_wd in naver_wt.intro[intro_num] :
        naver_wt_recommend.append(naver_wt.name[intro_num])
        naver_wt_url.append(naver_wt.url[intro_num])

# 다음 웹툰에서 입력 관련 컨텐츠 뽑기
for intro_num in range(len(daum_wt.intro)):
    if input_wd in daum_wt.intro[intro_num] :
        daum_wt_recommend.append(daum_wt.name[intro_num])
        daum_wt_url.append(daum_wt.url[intro_num])


# 네이버 웹소설에서 입력 관련 컨텐츠 뽑기
for intro_num in range(len(naver_nvl.intro)):
    if input_wd in naver_nvl.intro[intro_num] :
        naver_nvl_recommend.append(naver_nvl.name[intro_num])
        naver_nvl_url.append(naver_nvl.url[intro_num])


# 넷플릭스에서 입력 관련 컨텐츠 뽑기
for intro_num, intro in enumerate(netflix.netflix):
    if input_wd in intro :
        netflix_recommend.append(netflix.title[intro_num])
        netflix_url.append(netflix.link[intro_num])

print("추천 네이버 웹툰 : ", end='')
for i in range(len(naver_wt_recommend)) : 
    print(f"({naver_wt_recommend[i]},{naver_wt_url[i]})", end=' ')
print()
print("추천 다음 웹툰 : ", end='')
for i in range(len(daum_wt_recommend)) : 
    print(f"({daum_wt_recommend[i]},{daum_wt_url[i]})", end=' ')
print()
print("추천 네이버 웹소설 : ", end='')
for i in range(len(naver_nvl_recommend)) : 
    print(f"({naver_nvl_recommend[i]},{naver_nvl_url[i]})",end=' ')
print()
print("추천 넷플릭스 : ", end='')
for i in range(len(netflix_recommend)) : 
    print(f"({netflix_recommend[i]},{netflix_url[i]})",end=' ')


학교
추천 네이버 웹툰 : (인생존망,https://comic.naver.com/webtoon/list.nhn?titleId=733766&weekday=mon) (야자괴담,https://comic.naver.com/webtoon/list.nhn?titleId=743025&weekday=mon) (럭키언럭키,https://comic.naver.com/webtoon/list.nhn?titleId=736281&weekday=wed) (셧업앤댄스,https://comic.naver.com/webtoon/list.nhn?titleId=737032&weekday=thu) (커피도둑,https://comic.naver.com/webtoon/list.nhn?titleId=751492&weekday=sat) (소녀재판,https://comic.naver.com/webtoon/list.nhn?titleId=743838&weekday=sun) (학교정벌,https://comic.naver.com/webtoon/list.nhn?titleId=739097&weekday=sun) (신이 담긴 아이,https://comic.naver.com/webtoon/list.nhn?titleId=745652&weekday=sun) (만찢남녀,https://comic.naver.com/webtoon/list.nhn?titleId=733274&weekday=sun) (Here U Are,https://comic.naver.com/webtoon/list.nhn?titleId=721109&weekday=sun) 
추천 다음 웹툰 : (아싸가 알아버렸다,http://webtoon.daum.net/webtoon/view/noticingOutsider) (이대로 멈출 순 없다,http://webtoon.daum.net/webtoon/view/notstopping) (학교대표,http://webtoon.daum.net/webtoon/view/nege) (우리학교 김선생님,http://webtoon.daum.ne